<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [6]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [7]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [8]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [9]:
%%sql
SELECT *
FROM movies



 * sqlite:///movies.sqlite
Done.


IMDB_key,name,year,duration
tt0111161,The Shawshank Redemption,1994,142
tt0068646,The Godfather,1972,275
tt0468569,The Dark Knight,2008,152
tt0167260,The Lord of the Rings: The Return of the King,2003,301


+ Show the performance dates for one of the movies.

In [10]:
%%sql
SELECT start_date,start_time,th_name, name
FROM shows
LEFT JOIN movies
USING (IMDB_key)
WHERE name = "The Shawshank Redemption"


 * sqlite:///movies.sqlite
Done.


start_date,start_time,th_name,name
2019–02-12,20:00,SF LUND,The Shawshank Redemption
2019–02-13,20:00,SF LUND,The Shawshank Redemption
2019–02-13,20:00,SF STOCKHOLM,The Shawshank Redemption


+ Show all data concerning performances at a given theatere
  on a given date.

In [11]:
%%sql
SELECT *, name as title
FROM theaters
LEFT JOIN shows
USING (th_name)
JOIN movies
USING (IMDB_key)
WHERE th_name = "SF STOCKHOLM"

 * sqlite:///movies.sqlite
Done.


th_name,capacity,IMDB_key,start_date,start_time,name,year,duration,title
SF STOCKHOLM,300,tt0068646,2019–02-12,20:00,The Godfather,1972,275,The Godfather
SF STOCKHOLM,300,tt0111161,2019–02-13,20:00,The Shawshank Redemption,1994,142,The Shawshank Redemption
SF STOCKHOLM,300,tt0068646,2019–02-11,20:00,The Godfather,1972,275,The Godfather


+ List all customers

In [12]:
%%sql
SELECT *
FROM customers


 * sqlite:///movies.sqlite
Done.


user_id,full_name,password
3a694f22b05303fdc9e4a93a2f296913,FABIAN FRANKEL,hallojsan
ceedecff369b961c36edfeb3668b9988,SEPEHR TAYARI,jamboseli
8a876425243ea4a09ea9870ce196c474,MARCUS INGEMANSSON,DJINGSTRING


+ List all tickets

In [13]:
%%sql
SELECT *
FROM tickets


 * sqlite:///movies.sqlite
Done.


ticket_id,th_name,IMDB_key,start_time,start_date
da0a4cc3548c2993aa5c72835ccdb564,SF LUND,tt0111161,2019–02-12,20:00
6eb6d9a1b7b85da12ab5fe1edf8c1988,SF LUND,tt0111161,2019–02-13,20:00
cac6b5140981aac8b91700eec596e899,SF LUND,tt0068646,2019–02-11,20:00
c8555e5601096ab331e1ea431dfb1917,SF MALMO,tt0468569,2019–02-13,20:00
c961607f07c6de7620dc7d005cf16324,SF MALMO,tt0068646,2019–02-11,20:00
ff5facd0ddf80600046ae0775338e4a2,SF STOCKHOLM,tt0068646,2019–02-12,20:00
8b08557a18fbde6a59b173f7b556e3e0,SF STOCKHOLM,tt0111161,2019–02-13,20:00
ce5480dd27d4935e0aaf492990b5b1b5,SF STOCKHOLM,tt0068646,2019–02-11,20:00


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [14]:
%%sql
INSERT
INTO tickets (th_name, IMDB_key, start_time, start_date)
VALUES ("SF LUND", "tt0111161", "20:00", "2019–02-12")


 * sqlite:///movies.sqlite
1 rows affected.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2020), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [15]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid()

 * sqlite:///movies.sqlite
Done.


ticket_id
034fe1a4b42d9b5ee47598fb9e39ca69


+ Try to insert two movie theaters with the same name (this
  should fail).

In [16]:
%%sql
INSERT
INTO theaters
VALUES ("SF LUND", 250
);



 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.th_name
[SQL: INSERT
INTO theaters
VALUES ("SF LUND", 250
);]
(Background on this error at: http://sqlalche.me/e/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [17]:
%%sql
INSERT
INTO shows
VALUES ("tt0111161", "SF LUND", "2019–02-12", "20:00"
);


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: shows.IMDB_key, shows.start_time, shows.start_date, shows.th_name
[SQL: INSERT
INTO shows
VALUES ("tt0111161", "SF LUND", "2019–02-12", "20:00"
);]
(Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist.

In [18]:
%%sql
INSERT
    INTO tickets (IMDB_key, th_name, start_time, start_date)
    VALUES ("tt0111161", "SF LUND", "2019–02-12", "20:00"
);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
    INTO tickets (IMDB_key, th_name, start_time, start_date)
    VALUES ("tt0111161", "SF LUND", "2019–02-12", "20:00"
);]
(Background on this error at: http://sqlalche.me/e/gkpj)